Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

In [ ]:
data_lake_account_name = ''
file_system_name = ''

keyvault_name = ''

query = ''
topic = ''

In [ ]:
if topic == '':
    topic = query
subtopic = ""
query_language = "All"
target_languages = "English,Spanish"
num_tweets = 100
days = 7

language="en" 
maxdays=365
maxtweets_persearch=100

In [ ]:
from datetime import datetime, date, timedelta # don't import time here. It messes with the default library
base_path = f"abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/"
foldername_suffix = (datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
# News API
NEWS_API_KEY = TokenLibrary.getSecret(keyvault_name,"NEWSAPIKEY","KeyVaultLinkedService")
# Text analytics
TEXT_ANALYTICS_KEY = TokenLibrary.getSecret(keyvault_name,"TEXTANALYTICSKEY","KeyVaultLinkedService")
TEXT_ANALYTICS_ENDPOINT = TokenLibrary.getSecret(keyvault_name,"TEXTANALYTICSENDPOINT","KeyVaultLinkedService")
TEXT_ANALYTICS_REGION = TokenLibrary.getSecret(keyvault_name,"TEXTANALYTICSREGION","KeyVaultLinkedService")
# Translator
TRANSLATOR_KEY = TokenLibrary.getSecret(keyvault_name,"TRANSLATORKEY","KeyVaultLinkedService")
TRANSLATOR_ENDPOINT = TokenLibrary.getSecret(keyvault_name,"TRANSLATORENDPOINT","KeyVaultLinkedService")
TRANSLATOR_REGION = TokenLibrary.getSecret(keyvault_name,"TRANSLATORREGION","KeyVaultLinkedService")
# Twitter
TWITTER_API_KEY = TokenLibrary.getSecret(keyvault_name,"TWITTERAPIKEY","KeyVaultLinkedService")
TWITTER_API_SECRET_KEY = TokenLibrary.getSecret(keyvault_name,"TWITTERAPISECRETKEY","KeyVaultLinkedService")
TWITTER_ACCESS_TOKEN = TokenLibrary.getSecret(keyvault_name,"TWITTERACCESSTOKEN","KeyVaultLinkedService")
TWITTER_ACCESS_TOKEN_SECRET = TokenLibrary.getSecret(keyvault_name,"TWITTERACCESSTOKENSECRET","KeyVaultLinkedService")
TWITTER_API_AUTH = {
  'consumer_key': TWITTER_API_KEY,
  'consumer_secret': TWITTER_API_SECRET_KEY,
  'access_token': TWITTER_ACCESS_TOKEN,
  'access_token_secret': TWITTER_ACCESS_TOKEN_SECRET,
}
# Azure Maps
MAPS_KEY = TokenLibrary.getSecret(keyvault_name,"MAPSKEY","KeyVaultLinkedService")


In [ ]:
from pyspark.sql.functions import *

last_created_at = None

try:
    sql_str = 'select max(created_at) from tweets_data where query =  ' + '\'' + str(query) + '\''
    df_maxdate = spark.sql(sql_str).first()
    last_created_at = df_maxdate[0]
except:
    last_created_at = None


In [ ]:
import re
import sys
import copy
import tweepy
import urllib.parse
import sys, time, json, requests, uuid

from tweepy import API
from tweepy import Cursor
from tweepy import OAuthHandler

from dateutil.parser import parse
from datetime import datetime, date, timedelta # don't import time here. It messes with the default library


from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

def authenticate_client():
    """
    Returns: text analytics client
    """
    ta_credential = AzureKeyCredential(TEXT_ANALYTICS_KEY)
    text_analytics_client = TextAnalyticsClient(
            endpoint=TEXT_ANALYTICS_ENDPOINT, 
            credential=ta_credential)
    return text_analytics_client
text_analytics_client = authenticate_client()

def get_sentiment(inp_text):
    documents = [inp_text]
    response = text_analytics_client.analyze_sentiment(documents = documents)[0]  
    try:
        overallscore = response.confidence_scores.positive + (0.5*response.confidence_scores.neutral) # check logic of this
        return response.sentiment, overallscore
    except Exception as err:
        print("Encountered Sentiment exception. {}".format(err))
        return "Neutral",0.5
def get_ner(inp_text):
    try:
        documents = [inp_text]
        result = text_analytics_client.recognize_entities(documents = documents)[0]  
        return [{"text": x.text, "category": x.category, "subcategory": x.subcategory, "length": x.length, "offset": x.offset, "confidence_score": x.confidence_score} for x in result.entities]
    except Exception as err:
        print("Encountered NER exception. {}".format(err))
    return []
def get_key_phrases(inp_text):
    try:
      documents = [inp_text]
      response = text_analytics_client.extract_key_phrases(documents = documents)[0] 
      if not response.is_error:
          return response.key_phrases
      else:
          print(response.id, response.error)
    except Exception as err:
      print("Encountered Translation exception. {}".format(err))
    return []

# Tweet Entities processing
import requests
import aiohttp
import asyncio
import json

def get_maps_response(inp):
    url = "https://atlas.microsoft.com/search/fuzzy/json?&subscription-key="+MAPS_KEY+"&api-version=1.0&language=en-US&query="+inp
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()

def get_translation(inp_text, to_languages):
    """
    Params:
    inp_text: text to be translated
    to_languages: list of languages to translate to
    Returns: {lang_code: translation}, language code of the original text
    Call to translator cognitive service detects language and translates to the target languages. 
    Result is a dictionary of language codes to translated text, along with the language detected.
    """
    # Translator setup
    translator_path = "/translate"
    translator_url = TRANSLATOR_ENDPOINT + translator_path
    params = {
    "api-version": "3.0",
    "to": to_languages
    }
    headers = {
    'Ocp-Apim-Subscription-Key': TRANSLATOR_KEY,
    'Ocp-Apim-Subscription-Region': TRANSLATOR_REGION,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
    }
    # create and send request
    body = [{
    'text': inp_text
    }]
    request = requests.post(translator_url, params=params, headers=headers, json=body)
    response = request.json()
    
    try:
        from_language = response[0]["detectedLanguage"]["language"]
        translations = response[0]["translations"]
        res = {} 
        for trans in translations:
            res[trans['to']] = trans['text']
        
        return res, from_language 
    except Exception as err:
        print("Encountered an exception. {}".format(err))
        return err

In [ ]:
import re
import sys
import copy
import tweepy
import urllib.parse
import sys, time, json, requests, uuid

from tweepy import API
from tweepy import Cursor
from tweepy import OAuthHandler

from dateutil.parser import parse
from datetime import datetime, date, timedelta # don't import time here. It messes with the default library

from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

auth = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_SECRET_KEY)
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
auth_api = API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

def remove_emojis(data):
    emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", re.UNICODE)
    return re.sub(emoji, '', data)


In [ ]:
# https://developer.twitter.com/en/docs/twitter-api/v1/rules-and-filtering/search-operators - query reference

LANGUAGE_CODES={"All":"","Afrikaans":"af","Arabic":"ar","Assamese":"as","Bangla":"bn","Bosnian(Latin)":"bs","Bulgarian":"bg","Cantonese(Traditional)":"yue","Catalan":"ca","Chinese Simplified":"zh-Hans","Chinese Traditional":"zh-Hant","Croatian":"hr","Czech":"cs","Dari":"prs","Danish":"da","Dutch":"nl","English":"en","Estonian":"et","Fijian":"fj","Filipino":"fil","Finnish":"fi","French":"fr","German":"de","Greek":"el","Gujarati":"gu","Haitian Creole":"ht","Hebrew":"he","Hindi":"hi","Hmong Daw":"mww","Hungarian":"hu","Icelandic":"is","Indonesian":"id","Irish":"ga","Italian":"it","Japanese":"ja","Kannada":"kn","Kazakh":"kk","Klingon":"tlh-Latn","Klingon(plqaD)":"tlh-Piqd","Korean":"ko","Kurdish(Central)":"ku","Kurdish(Northern)":"kmr","Latvian":"lv","Lithuanian":"lt","Malagasy":"mg","Malay":"ms","Malayalam":"ml","Maltese":"mt","Maori":"mi","Marathi":"mr","Norwegian":"nb","Odia":"or","Pashto":"ps","Persian":"fa","Polish":"pl","Portuguese(Brazil)":"pt-br","Portuguese(Portugal)":"pt-pt","Punjabi":"pa","Queretaro Otomi":"otq","Romanian":"ro","Russian":"ru","Samoan":"sm","Serbian(Cyrillic)":"sr-Cyrl","Serbian(Latin)":"sr-Latn","Slovak":"sk","Slovenian":"sl","Spanish":"es","Swahili":"sw","Swedish":"sv","Tahitian":"ty","Tamil":"ta","Telugu":"te","Thai":"th","Tongan":"to","Turkish":"tr","Ukrainian":"uk","Urdu":"ur","Vietnamese":"vi","Welsh":"cy","Yucatec Maya":"yua"}
topic = topic.lower()
query_language = LANGUAGE_CODES.get(query_language, "")
target_languages = [LANGUAGE_CODES.get(lang, "") for lang in target_languages.split(",")]
if "en" not in target_languages:
  target_languages.append("en") # we always include english in target languages
num_tweets = int(num_tweets)
max_days = int(days)

regions = ["Africa","Arabian Gulf","Asia","Central America","Europe","Middle East","North America","Oceania","South America"]


end_date = datetime.utcnow() - timedelta(days=maxdays)
all_tweets = []
all_users = []
all_hashtags = []
all_urls = []
all_media = []
all_handles = []
count = 0
query_search = True

dt2 = datetime.now()
ts= int(time.mktime(dt2.timetuple()))
at=dt2.strftime("%m/%d/%Y, %H:%M:%S %Z")

for status in Cursor(auth_api.search, q=query, lang=language, result='recent', tweet_mode = "extended", include_rts='False').items(maxtweets_persearch):

    tweet = {}
    user = {}
    tweet_obj = status._json
    
    if (last_created_at != None and last_created_at != '' and parse(tweet_obj["created_at"]).timestamp() <= last_created_at.timestamp()):
        #if (parse(tweet_obj["created_at"]).timestamp() <= last_created_at.timestamp()): #ignore already loaded records
        continue

    user_obj = tweet_obj["user"]
    user["topickey"] = topic
    user["id"] = user_obj["id_str"]
    user["document_type"] = "user"
    user['inserted_at'] = at
    user['inserted_ts'] = ts
    user['name'] = user_obj["name"]
    user['screen_name'] = user_obj["screen_name"]
    user['location'] = user_obj["location"]
    user['description'] = user_obj["description"]
    user['url'] = user_obj["url"]
    user['protected'] = user_obj["protected"]
    user['followers_count'] = user_obj["followers_count"]
    user['friends_count'] = user_obj["friends_count"]
    user['listed_count'] = user_obj["listed_count"]
    user['favourites_count'] = user_obj["favourites_count"]
    user['profile_image_url'] = user_obj["profile_image_url"]
    user['profile_image_url_https'] = user_obj["profile_image_url_https"]
    user['geo_enabled'] = user_obj["geo_enabled"]
    user['verified'] = user_obj["verified"]
    user['following'] = user_obj["following"]
    user['created_at'] = user_obj["created_at"]

    user["month_year"] = str(str(parse(user_obj["created_at"]).month) + "_"+str(parse(user_obj["created_at"]).year))
    user["country_azuremaps"] = ''
    user["country_code_azuremaps"] = ''
    user_location = tweet_obj["user"]["location"]
    if user_location != "" and user_location not in regions:
        r_json = get_maps_response(user_location)
        if r_json: # i.e. got a response
            if r_json["summary"]["numResults"] > 0:
                # there is a location detected, so get the country
                if "address" in r_json['results'][0].keys():
                    top_match = r_json['results'][0]["address"]
                    if "country" in top_match.keys() and "countryCode" in top_match.keys() :
                        country = top_match["country"]
                        country_code = top_match["countryCode"]
                        user["country_azuremaps"] = country
                        user["country_code_azuremaps"] = country_code

    id_str = tweet_obj["id_str"]

    if tweet_obj['entities'] is not None:
        for key, value in tweet_obj['entities'].items():
            if key == 'hashtags':
                for h in value:
                    hashtag = {}
                    hashtag['id'] = id_str
                    hashtag['text'] = h['text']
                    hashtag['created_datetime'] = datetime.now()
                    all_hashtags.append(hashtag)

    for um in tweet_obj['entities']['user_mentions']:
        user_mention = {}
        user_mention['id'] = id_str
        user_mention['screen_name'] = um['screen_name']
        user_mention['created_datetime'] = datetime.now()
        all_handles.append(um)
    for u in tweet_obj['entities']['urls']:
        urls = {}
        urls['id'] = id_str
        urls['url'] = u['url']
        urls['expanded_url'] = u['expanded_url']
        urls['display_url'] = u['display_url']
        urls['created_datetime'] = datetime.now()
        all_urls.append(urls)
    if 'media' in tweet_obj['entities']:
        for m in tweet_obj['entities']['media']:
            media = {}
            media['id'] = id_str
            media['media_url'] = m['media_url']
            media['created_datetime'] = datetime.now()
            all_media.append(media)

    tweet["userid"]=user["id"]

    dt2 = datetime.now()
    ts = int(time.mktime(dt2.timetuple()))
    at = dt2.strftime("%m/%d/%Y, %H:%M:%S %Z")
    tweet['inserted_at'] = at
    tweet['inserted_ts'] = ts
    tweet["originalid"] = tweet_obj["id"]
    tweet["id"] = str(int(tweet_obj["id_str"])+abs(hash(topic))) # artifically creating our own ID
    tweet["topickey"] = topic
    tweet["subtopic"] = subtopic
    tweet["created_at"] = parse(tweet_obj["created_at"])
    tweet["created_date"] = parse(tweet_obj["created_at"]).date()
    tweet["month_year"] = str(str(parse(tweet_obj["created_at"]).month) + "_"+str(parse(tweet_obj["created_at"]).year))

    tmp_text = tweet_obj["full_text"].replace('\n','. ').replace('\r','.').replace('..','. ').replace(',.','. ').replace(';.','. ').replace('?.','. ').replace('!.','. ').replace(':.','. ').lstrip('.').lstrip(' ')
    tmp_text = remove_emojis(tmp_text)
    tweet["text"]= tmp_text
    tweet["document_type"] = "tweet"

    tweet["search_type"]='Topic Search'  
    tweet["query"] = str(query)

    tweet["is_quote_status"] = tweet_obj["is_quote_status"]
    tweet["retweet_count"] = tweet_obj["retweet_count"]
    tweet["favorite_count"] = tweet_obj["favorite_count"]
    tweet["favorited"] = tweet_obj["favorited"]
    tweet["retweeted"] = tweet_obj["retweeted"]
    tweet["lang"] = tweet_obj["lang"]
    tweet["source"] = tweet_obj["source"]
    
    city = 'NA'
    country = 'NA'  
    if tweet_obj['place'] is None:
        city = 'NA'
        country = 'NA'  
    else:
      city = tweet_obj["place"]['name']
      country = tweet_obj["place"]['country']
    
    tweet['city'] = city
    tweet['country'] = country

    all_tweets.append(tweet)
    all_users.append(user)
    count +=1
    if count > num_tweets:
        break

In [ ]:
from pyspark.sql import Row

df_tweets = spark.createDataFrame(Row(**x) for x in all_tweets)
file_path = base_path + 'TweetsData/' + str(query) + '_' + str(topic) + foldername_suffix
df_tweets.write.format('json').save(file_path)
df_tweets.write.mode("append").saveAsTable("tweets_data")

df_users = spark.createDataFrame(Row(**x) for x in all_users)
file_path = base_path + 'TweetsUsersData/' +str(query) + '_' + str(topic) + foldername_suffix
df_users.write.format('json').save(file_path)
df_users = df_users.dropDuplicates(['id'])
df_users.write.mode("append").saveAsTable("tweets_users_data")

df_hashtags = spark.createDataFrame(Row(**x) for x in all_hashtags)
file_path = base_path + 'TweetsHashtagData/' + str(query) + '_' + str(topic) + foldername_suffix
df_hashtags.write.format('json').save(file_path)
df_hashtags.write.mode("append").saveAsTable("tweets_hashtag_data")

df_urls = spark.createDataFrame(Row(**x) for x in all_urls)
file_path = base_path + 'TweetsURLData/' + str(query) + '_' + str(topic) + foldername_suffix
df_urls.write.format('json').save(file_path)
df_urls.write.mode("append").saveAsTable("tweets_url_data")

df_media = spark.createDataFrame(Row(**x) for x in all_media)
file_path = base_path + 'TweetsMediaData/' + str(query) + '_' + str(topic) + foldername_suffix
df_media.write.format('json').save(file_path)
df_media.write.mode("append").saveAsTable("tweets_media_data")

df_handles = spark.createDataFrame(Row(**x) for x in all_handles)
file_path = base_path + 'TweetsHandlesData/' + str(query) + '_' + str(topic) + foldername_suffix
df_handles.write.format('json').save(file_path)
df_handles.write.mode("append").saveAsTable("tweets_handles_data")

In [ ]:
import ast
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

def get_inserted_date(publishedAt):
    dt2 = datetime.now()
    ts= int(time.mktime(dt2.timetuple()))
    at=dt2.strftime("%m/%d/%Y, %H:%M:%S %Z")
    return(at)
inserted_date_udf = udf(lambda publishedAt: get_inserted_date(publishedAt), returnType=StringType())

schema = StructType([
    StructField("Translations", MapType(StringType(),StringType()), False),
    StructField("QueryLanguage", StringType(), False)])

def get_translations(originaltext,target_languages): 
    translations_text, query_language = get_translation(originaltext, ast.literal_eval(target_languages))
    return Row('Translations', 'QueryLanguage')(translations_text, query_language)

get_translations_udf = udf(lambda originaltext,target_languages: get_translations(originaltext,target_languages),returnType=schema)


df_translations = df_tweets.select(['id','text']) \
                .withColumn('Ttext1', get_translations_udf(col("text"),lit(str(target_languages)))) \
                .withColumn('original_language', col("Ttext1.QueryLanguage")) \
                .select(explode(col("Ttext1.Translations")).alias("language","TText"),'id','text','original_language') \
                .withColumn('created_datetime', inserted_date_udf(col("id")))

df_translations = df_translations.select('id','text','TText','language','original_language','created_datetime')
df_translations.write.mode("append").saveAsTable("tweets_translations")

In [ ]:
from pyspark.sql import Row

schema = MapType(StringType(),ArrayType(MapType(StringType(),StringType())))
        
def get_ners(original_language,language,content,TContent):
    named_entities = []
    if language != "en":
        if TContent !='':
            named_entities = get_ner(content)
            named_entity_obj = {language: named_entities}
        else:
            named_entity_obj = {language: named_entities}
    else:
        named_entities = get_ner(content) 
        named_entity_obj = {"en": named_entities}
    return(named_entity_obj)

get_ner_udf = udf(lambda original_language,language,content,TContent: get_ners(original_language,language,content,TContent),returnType=schema)

df_ner = df_translations \
                .select(['id','original_language','language','text','TText']) \
                .withColumn('NER', get_ner_udf(col("original_language"),col("language"),col("text"),col("TText"))) \
                .withColumn('created_datetime', inserted_date_udf(col("id"))) \
                .select(explode(col("NER")).alias("language","ner1"),'id','original_language','created_datetime') \
                .select(explode(col("ner1")).alias("ner2"),'id','created_datetime','language','original_language') \
                .select(['id','created_datetime','language','original_language',col("ner2.text").alias("value"),col("ner2.subcategory").alias("subcategory"),col("ner2.offset").alias("offset"),
                col("ner2.confidence_score").alias("confidence_score"), col("ner2.category").alias("category"),col("ner2.length").alias("length")])
df_ner = df_ner.select('id','language','original_language','created_datetime','value','category','subcategory','offset','confidence_score','length')
df_ner.write.mode("append").saveAsTable("tweets_entities")

In [ ]:
import ast
from pyspark.sql import Row

schema = StructType([
    StructField("sentiment", StringType(), False),
    StructField("sentiment_score", DecimalType(), False)])

def get_sentiments(original_language,title,TText): 
    if original_language != "en":
        sentiment, sentiment_score = get_sentiment(TText)
    else:
        sentiment, sentiment_score = get_sentiment(title)
    return Row('sentiment', 'sentiment_score')(sentiment, sentiment_score)

get_sentiments_udf = udf(lambda originaltext,title,TText: get_sentiments(originaltext,title,TText),returnType=schema)

df_sentiments = df_translations \
                .select(['id','original_language','text','TText']) \
                .withColumn('TS1', get_sentiments_udf(col("original_language"),col("text"),col("TText"))) \
                .withColumn('sentiment', col("TS1.sentiment")) \
                .withColumn('sentiment_score', col("TS1.sentiment_score")) \
                .withColumn('created_datetime', inserted_date_udf(col("id"))) \
                .select(['id','sentiment','sentiment_score','created_datetime'])
df_sentiments.write.mode("append").saveAsTable("tweets_sentiments")